# NISQ Experiment

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from qiskit import Aer
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from qiskit.providers.aer import AerSimulator
from copy import deepcopy

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from utils import *
from experiments import *
from qiskit.test.mock.backends import FakeCasablanca
#np.set_printoptions(threshold=sys.maxsize)

In [2]:
#qk.IBMQ.save_account("66718f8f8aef22bcb6ebe86ad94a11f1fd1f4c55100829bb13f16e6b448e0a1ec6d09c459d738f58d0cbd8398a2a1f5e185a4706a61b6f896a5ce2983e136429", overwrite=True) 
provider = qk.IBMQ.load_account()
provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend("ibmq_manila")
#backend = FakeCasablanca()
#backend = AerSimulator()

In [3]:
def variational_circuit(n):
    theta = np.random.uniform(-np.pi, np.pi, 4*n)
    circuit = qk.QuantumCircuit(n)
    for i, angle in enumerate(theta[:n]):
        circuit.ry(angle, i)
    
    for i, angle in enumerate(theta[n:2*n]):
        circuit.crx(angle, i, (i+1)%n)
        
    #for i, angle in enumerate(theta[2*n:3*n]):
    #    circuit.ry(angle, i)
        
    #for i, angle in enumerate(theta[3*n:]):
    #    circuit.crx(angle, (n-i)%n, n-i-1)
    
    return circuit

## Pauli String Expectation Values

## Two Qubit POVM

In [4]:
n = 2
d = 2**n
#backend = AerSimulator()
backend = FakeCasablanca()
np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n).reverse_bits()
print(circuit_target)

N = 324
input_list, circuit_list = generate_pauli_circuits(circuit_target, N, trace=False)
    
for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        circuit = circuit.compose(circuit_target)
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)

for i in range(d):
    circuit_list.extend(generate_bitstring_circuits(n))
    
    
job = qk.execute(circuit_list, backend, shots = 20000, optimization_level = 0)
result = job.result()


counts_list = [result.get_counts(circuit) for circuit in circuit_list]

data = [input_list, counts_list]

pickle.dump(data, open("../../data/twoQubits_expectation_POVM.p", "wb"))

      ┌────────────┐ ┌────────────┐               
q_0: ─┤ Ry(2.8319) ├─┤ Rx(1.4577) ├───────■───────
     ┌┴────────────┴┐└─────┬──────┘┌──────┴──────┐
q_1: ┤ Ry(-0.78829) ├──────■───────┤ Rx(0.61989) ├
     └──────────────┘              └─────────────┘


In [6]:
n = 3
d = 2**n
#backend = AerSimulator()
backend = FakeCasablanca()
np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n).reverse_bits()
print(circuit_target)

N = 5000
input_list, circuit_list = generate_pauli_circuits(circuit_target, N, trace=False)
    
for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        circuit = circuit.compose(circuit_target)
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)

for i in range(d):
    circuit_list.extend(generate_bitstring_circuits(n))
    
    
job = qk.execute(circuit_list, backend, shots = 20000, optimization_level = 0)
result = job.result()


counts_list = [result.get_counts(circuit) for circuit in circuit_list]

data = [input_list, counts_list]

pickle.dump(data, open("../../data/threeQubits_expectation_POVM.p", "wb"))

      ┌────────────┐                ┌─────────────┐               
q_0: ─┤ Ry(1.4577) ├────────────────┤ Rx(-2.1613) ├───────■───────
      ├────────────┤ ┌─────────────┐└──────┬──────┘       │       
q_1: ─┤ Ry(2.8319) ├─┤ Rx(0.61989) ├───────■──────────────┼───────
     ┌┴────────────┴┐└──────┬──────┘               ┌──────┴──────┐
q_2: ┤ Ry(-0.78829) ├───────■──────────────────────┤ Rx(-2.1615) ├
     └──────────────┘                              └─────────────┘


In [ ]:
n = 5
d = 2**n
#backend = AerSimulator()
backend = FakeCasablanca()
np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n).reverse_bits()
print(circuit_target)

N = 10000
input_list, circuit_list = generate_pauli_circuits(circuit_target, N, trace=False)
    
for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        circuit = circuit.compose(circuit_target)
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)

for i in range(d):
    circuit_list.extend(generate_bitstring_circuits(n))

#circuit_list_ = []
#for circuit in circuit_list:
#    circuit_list_.append(circuit.reverse_bits())
    
#circuit_list = circuit_list_
    
    
job = qk.execute(circuit_list, backend, shots = 20000, optimization_level = 0)
result = job.result()


counts_list = [result.get_counts(circuit) for circuit in circuit_list]

data = [input_list, counts_list]

pickle.dump(data, open("../../data/fiveQubits_expectation_POVM.p", "wb"))

## Two Qubit POVM, two blocks

In [7]:
n = 2
d = 2**n
#backend = AerSimulator()
backend = FakeCasablanca()
np.random.seed(42)
random.seed(42)

circuit_target1 = variational_circuit(n).reverse_bits()
circuit_target2 = variational_circuit(n).reverse_bits()
circuit_target3 = circuit_target1.compose(circuit_target2)
print(circuit_target1)
print(circuit_target2)

N = 324
input_list1, circuit_list1 = generate_pauli_circuits(circuit_target1, N, trace=False)
input_list2, circuit_list2 = generate_pauli_circuits(circuit_target2, N, trace=False)
 
input_list = []
input_list.extend(input_list1)
input_list.extend(input_list2)

circuit_list = []
circuit_list.extend(circuit_list1)
circuit_list.extend(circuit_list2)

    
q_reg = qk.QuantumRegister(n)
c_reg = qk.ClassicalRegister(n)
circuit = qk.QuantumCircuit(q_reg, c_reg)

circuit = circuit.compose(circuit_target3)

circuit.measure(q_reg, c_reg)
circuit_list.append(circuit)


for i in range(d):
    circuit_list.extend(generate_bitstring_circuits(n))
    
    
job = qk.execute(circuit_list, backend, shots = 20000, optimization_level = 0)
result = job.result()


counts_list = [result.get_counts(circuit) for circuit in circuit_list]

data = [input_list, counts_list]

pickle.dump(data, open("..\..\data\twoQubits_twoBlocks.p", "wb"))

      ┌────────────┐ ┌────────────┐               
q_0: ─┤ Ry(2.8319) ├─┤ Rx(1.4577) ├───────■───────
     ┌┴────────────┴┐└─────┬──────┘┌──────┴──────┐
q_1: ┤ Ry(-0.78829) ├──────■───────┤ Rx(0.61989) ├
     └──────────────┘              └─────────────┘
      ┌────────────┐┌─────────────┐              
q_0: ─┤ Ry(1.3074) ├┤ Rx(-3.0123) ├──────■───────
     ┌┴────────────┤└──────┬──────┘┌─────┴──────┐
q_1: ┤ Ry(0.63532) ├───────■───────┤ Rx(2.9525) ├
     └─────────────┘               └────────────┘


## Three Qubit POVM, two blocks

In [9]:
n = 3
d = 2**n
#backend = AerSimulator()
backend = FakeCasablanca()
np.random.seed(42)
random.seed(42)

circuit_target1 = variational_circuit(n).reverse_bits()
circuit_target2 = variational_circuit(n).reverse_bits()
circuit_target3 = circuit_target1.compose(circuit_target2)
print(circuit_target1)
print(circuit_target2)

N = 1000
input_list1, circuit_list1 = generate_pauli_circuits(circuit_target1, N, trace=False)
input_list2, circuit_list2 = generate_pauli_circuits(circuit_target2, N, trace=False)
 
input_list = []
input_list.extend(input_list1)
input_list.extend(input_list2)

circuit_list = []
circuit_list.extend(circuit_list1)
circuit_list.extend(circuit_list2)

    
q_reg = qk.QuantumRegister(n)
c_reg = qk.ClassicalRegister(n)
circuit = qk.QuantumCircuit(q_reg, c_reg)

circuit = circuit.compose(circuit_target3)

circuit.measure(q_reg, c_reg)
circuit_list.append(circuit)


for i in range(d):
    circuit_list.extend(generate_bitstring_circuits(n))
    
    
job = qk.execute(circuit_list, backend, shots = 20000, optimization_level = 0)
result = job.result()


counts_list = [result.get_counts(circuit) for circuit in circuit_list]

data = [input_list, counts_list]

pickle.dump(data, open("../../data/threeQubits_twoBlocks.p", "wb"))

      ┌────────────┐                ┌─────────────┐               
q_0: ─┤ Ry(1.4577) ├────────────────┤ Rx(-2.1613) ├───────■───────
      ├────────────┤ ┌─────────────┐└──────┬──────┘       │       
q_1: ─┤ Ry(2.8319) ├─┤ Rx(0.61989) ├───────■──────────────┼───────
     ┌┴────────────┴┐└──────┬──────┘               ┌──────┴──────┐
q_2: ┤ Ry(-0.78829) ├───────■──────────────────────┤ Rx(-2.1615) ├
     └──────────────┘                              └─────────────┘
     ┌─────────────┐               ┌───────────┐               
q_0: ┤ Ry(-1.9992) ├───────────────┤ Rx(-1.23) ├───────■───────
     ├─────────────┤┌─────────────┐└─────┬─────┘       │       
q_1: ┤ Ry(-1.8074) ├┤ Rx(-1.9892) ├──────■─────────────┼───────
     └┬────────────┤└──────┬──────┘             ┌──────┴──────┐
q_2: ─┤ Ry(2.0888) ├───────■────────────────────┤ Rx(0.15555) ├
      └────────────┘                            └─────────────┘
